In [26]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

In [7]:
url = 'http://books.toscrape.com/'
resp = requests.get(url)
resp

<Response [200]>

In [6]:
soup = bs(resp.text)

In [8]:
soup.find('title').text.strip()

'All products | Books to Scrape - Sandbox'

In [9]:
bks_tag = soup.find_all('article', class_='product_pod')

bk_tag = bks_tag[0]
bk_tag

<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">Â£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

In [11]:
ttl_tag = soup.find('a', title = True)
ttl_tag

<a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a>

In [12]:
ttl_tag.text

'A Light in the ...'

In [13]:
ttl_tag['title']

'A Light in the Attic'

In [14]:
bk_tag.find('p')['class'][1]

'Three'

In [15]:
bk_tag.find('p', class_ = 'price_color').text[1:]

'£51.77'

In [16]:
link = 'http://books.toscrape.com/' + bk_tag.find('a')['href']
link

'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'

In [38]:
def get_details(book_tag):
    title = book_tag.find('a', title = True)['title']
    rating = book_tag.find('p')['class'][1]
    price = book_tag.find('p', class_ = 'price_color').text[1:]
    link = 'http://books.toscrape.com/' + book_tag.find('a')['href']
    
    return title, rating, price, link

In [39]:
def get_soup(url):
    resp = requests.get(url)
    if resp.status_code==200:
        return bs(resp.text)
    else:
        return None
    
    
def get_books(url):
    soup = get_soup(url)
    book_tags = soup.find_all('article', class_='product_pod')
    
    books = [get_details(book_tag) for book_tag in book_tags]
    return books

In [40]:
url = 'http://books.toscrape.com/'
books = get_books(url)
len(books)

20

In [41]:
books[:2]

[('A Light in the Attic',
  'Three',
  '£51.77',
  'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'),
 ('Tipping the Velvet',
  'One',
  '£53.74',
  'http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html')]

In [42]:
def get_all_books(page =3):
    books = []
    for i in range(1,page+1):
        url = f'http://books.toscrape.com/catalogue/page-{i}.html'
        soup = get_soup(url)
        if soup:
            book_tags = soup.find_all('article', class_ = 'product_pod')
            
            for book_tag in book_tags:
                books.append(get_details(book_tag))
                
    books = pd.DataFrame(books, columns = ['title', 'rating', 'price', 'link'])
    return books

In [44]:
get_all_books(10).head()

,title,rating,price,link
0,A Light in the Attic,Three,£51.77,http://books.toscrape.com/a-light-in-the-attic...
1,Tipping the Velvet,One,£53.74,http://books.toscrape.com/tipping-the-velvet_9...
2,Soumission,One,£50.10,http://books.toscrape.com/soumission_998/index...
3,Sharp Objects,Four,£47.82,http://books.toscrape.com/sharp-objects_997/in...
4,Sapiens: A Brief History of Humankind,Five,£54.23,http://books.toscrape.com/sapiens-a-brief-hist...


### Final Scraping

In [102]:
def get_soup(url):
    try:
        resp = requests.get(url)
    except:
        return None
    
    if str(resp)=='<Response [200]>':
        return bs(resp.text)
    else:
        print(resp, bs(resp.text))
        return None
    
def get_details(book_tag):
#   title
    try:
        title = book_tag.find('a', title = True)['title']
    except:
        title = None
        
#     rating
    try:
        rating = book_tag.find('p')['class'][1]
        
    except:
        rating = None
        
#     price
    try:
        price  = book_tag.find('p', class_ = 'price_color').text[1:]
        
    except:
        price = None
        
#     link
    try:
        link = 'http://books.toscrape.com/' + book_tag.find('a')['href']
    except:
        link = None
        
    return title, rating, price, link

def get_all_books(page=3):
    books = []
    for i in range(1,page+1):
        url = f'http://books.toscrape.com/catalogue/page-{i}.html'
        soup = get_soup(url)
        if soup:
            try:
                book_tags = soup.find_all('article', class_ = 'product_pod')
                
                for book_tag in book_tags:
                    books.append(get_details(book_tag))
            except:
                print(f'Error reading page {i} . . .')

            time.sleep(1) # sleep before making the next request

    books = pd.DataFrame(books, columns=['title', 'rating', 'price', 'link'])
    return books

In [105]:
df = get_all_books(20)
# df.info

In [106]:
df

,title,rating,price,link
0,A Light in the Attic,Three,£51.77,http://books.toscrape.com/a-light-in-the-attic...
1,Tipping the Velvet,One,£53.74,http://books.toscrape.com/tipping-the-velvet_9...
2,Soumission,One,£50.10,http://books.toscrape.com/soumission_998/index...
3,Sharp Objects,Four,£47.82,http://books.toscrape.com/sharp-objects_997/in...
4,Sapiens: A Brief History of Humankind,Five,£54.23,http://books.toscrape.com/sapiens-a-brief-hist...
...,...,...,...,...
395,Take Me Home Tonight (Rock Star Romance #3),Three,£53.98,http://books.toscrape.com/take-me-home-tonight...
396,Sleeping Giants (Themis Files #1),One,£48.74,http://books.toscrape.com/sleeping-giants-them...
397,"Setting the World on Fire: The Brief, Astonish...",Two,£21.15,http://books.toscrape.com/setting-the-world-on...
398,Playing with Fire,Three,£13.71,http://books.toscrape.com/playing-with-fire_60...
